In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

In [2]:
data_path = "/home/akash/projects/AML/ds_comptetions/kaggle_sales_predict/data/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print onlyfiles

['shops.csv', 'test.csv', 'submission.csv', 'sales_train.csv', 'item_categories.csv', 'sample_submission.csv', 'submission_1.167.csv', 'items.csv']


In [3]:
categories = pd.read_csv(data_path+"item_categories.csv")

In [4]:
categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [5]:
categories.shape

(84, 2)

In [6]:
items = pd.read_csv(data_path+"items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
items.shape

(22170, 3)

In [8]:
shops = pd.read_csv(data_path+"shops.csv")

In [9]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [10]:
shops.shape

(60, 2)

In [11]:
sales = pd.read_csv(data_path+"sales_train.csv")
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [12]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [13]:
sales.shape

(2935849, 6)

In [14]:
test = pd.read_csv(data_path+"test.csv")

In [15]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [16]:
test.shape

(214200, 3)

In [17]:
sample_submit = pd.read_csv(data_path+"sample_submission.csv")
sample_submit.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


### Create dumb benchmarks

In [18]:
test_all_zeroes = test[["ID"]] # if we take only a single ssuare brackets we get a series not a DF
test_all_zeroes["item_cnt_month"] = 0
test_all_zeroes.to_csv("all_0.csv",index=False)

/home/akash/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
test_all_zeroes.head()

,ID,item_cnt_month
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


### all_0 benchmark = 1.25

In [20]:
test_all_ones = test[["ID"]]
test_all_ones["item_cnt_month"] = 1
test_all_ones.to_csv("all_1.csv",index=False)

/home/akash/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### all_1s becnhmark = 1.41214

* score_last_month = 1.167
* all_0s = 1.25
* all_1s = 1.41214

In [21]:
sales.loc[sales["item_cnt_day"]==0].shape

(0, 6)

#### All 0 has a score higher than all_1s and not much worse tha last month sales. This inidcates there are shop_id , item_id, pairs in test which did not sell even one item. Thus model should be able to predict 0.  

#### As we see below in sales there are no records for 0 sales. Sales only has a record if there was sale So if we create a grouped dataframe grouped by shop_id, item_id, month we wont get training data for 0 sales. However these are present in testTo allow the model to predict 0 we can add shop, item pairs to train data for which there was no sale i.e 0 sales. So model can learn to predict 0 sale. 

#### Thus we work wih a grid of all shop, item, pairs.



In [ ]:
from itertools import product
month = 0
shops_month_0 = sales[sales["date_block_num"] == month]["shop_id"].unique()
items_month_0 = sales[sales["date_block_num"] == month]["item_id"].unique()
num_shops_month_0 = len(shops_month_0)
num_items_month_0 = len(items_month_0)
print "num shops month 0 :%d"%num_shops_month_0
print "num items month 0 %d"%num_items_month_0
print "total pairs: %d" %(num_shops_month_0*num_items_month_0)
# * does the cross product and flattens. [month] is to create a iterable, as month is just an integer.
# list() converts the iterable to al list
all_pairs_month_0 = list(product(*[shops_month_0, items_month_0, [0]]))
print len(all_pairs_month_0)
print all_pairs_month_0[0:10]
print np.array(all_pairs_month_0[0:10])

In [ ]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

In [ ]:
# grid is a list containing oen entry for each month
print len(grid)
print grid[0].shape
total = 0
for i in range(34):
    total+= grid[i].shape[0]
print total

#### convert grid into data_frame

In [ ]:
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
print grid.shape

In [ ]:
grid.head()

#### Create grouped sales df

In [ ]:
#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'monthly_sale':'sum'}})
gb.head()

In [ ]:
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
gb.head()

#### Merge grid and grouped sales so that we have one entry for each triplet in grid. Fill missing entries in sales with 0

In [ ]:
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [ ]:
all_data.head()

In [ ]:
all_data.shape

In [ ]:
np.round(all_data.monthly_sale.mean(),decimals=4)

### Last Month Transformer

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin

class LastMonthSaleTransformer(BaseEstimator, RegressorMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        X_last = X.loc[(X["date_block_num"]==33) & (X["monthly_sale"]>0)]
        last_month_dict = X_last.groupby(["shop_id", "item_id"])["monthly_sale"].apply(list).to_dict()
        self.last_month_dict_ = last_month_dict
        return self
    
    def transform(self, X, y=None):
        last_month_dict = self.last_month_dict_
        
        def fill_last_month(key):
            if key in last_month_dict:
                return last_month_dict[key][0]
            else:
                return np.NaN
        
        X["last_month_sales"] = X.apply( lambda row: fill_last_month((row["shop_id"], row["item_id"])) , axis=1)
        X["last_month_sales"] = X["last_month_sales"].fillna(0)
        return X

In [ ]:
last_month_transformer = LastMonthSaleTransformer()
last_month_transformer.fit(all_data)
all_data = last_month_transformer.transform(all_data)

In [ ]:
print len(last_month_transformer.last_month_dict_)

In [ ]:
all_data.head()

In [ ]:
all_data.loc[all_data["date_block_num"] == 33]

In [ ]:
last_month_transformer.last_month_dict_[(2,486)]

In [ ]:
test = last_month_transformer.transform(test)

In [ ]:
test.head()

In [ ]:
submit_last_month = test[["ID","last_month_sales"]]
submit_last_month["last_month_sales"] = submit_last_month["last_month_sales"].clip(0,20)
submit_last_month = submit_last_month.rename(columns={"last_month_sales":"item_cnt_month"})
submit_last_month.to_csv("submit_last_month.csv",index=False)

### Mean encodings

In [ ]:
tmp = all_data[(all_data["date_block_num"]==33) & (all_data["monthly_sale"]>0)]
print tmp.groupby(["shop_id"])["monthly_sale"].agg(['sum','mean']).to_dict(orient="index")


In [ ]:
class LastMonthShopSaleTransformer(BaseEstimator, RegressorMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        X_last = X.loc[(X["date_block_num"]==33) & (X["monthly_sale"]>0)]
        self.last_month_sales_dict_ = X_last.groupby(["shop_id"])["monthly_sale"].agg(['sum','mean']).to_dict(orient="index")
        return self
    
    def transform(self, X, y=None):
        last_month_shop_mean = np.zeros(X.shape[0])
        last_month_shop_total = np.zeros(X.shape[0])
        
        i = 0 
        for idx, row in X.iterrows():
            shop_id = row["shop_id"]
            if shop_id in self.last_month_sales_dict_:
                last_month_shop_mean[i] = self.last_month_sales_dict_[shop_id]['mean']
                last_month_shop_total[i] = self.last_month_sales_dict_[shop_id]['sum']
            i+=1
            
        X["last_month_shop_mean"] = last_month_shop_mean
        X["last_month_shop_total"] = last_month_shop_total
    
        return X

In [ ]:
last_month_shop_sale_transformer = LastMonthShopSaleTransformer()
last_month_shop_sale_transformer.fit(all_data)

In [ ]:
all_data = last_month_shop_sale_transformer.transform(all_data)

In [ ]:
 all_data.head()

In [ ]:
print all_data[(all_data["shop_id"]==31) & (all_data["date_block_num"]==33)]["monthly_sale"].sum()
print all_data[(all_data["shop_id"]==31) & (all_data["date_block_num"]==33)]["monthly_sale"].mean()

In [ ]:
all_data[(all_data["shop_id"]==31) & (all_data["date_block_num"]==33)].head()

In [ ]:
test = last_month_shop_sale_transformer.transform(test)

In [ ]:
test.head()

In [ ]:
class LastMonthItemSaleTransformer(BaseEstimator, RegressorMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        X_last = X.loc[(X["date_block_num"]==33) & (X["monthly_sale"]>0)]
        self.last_month_sales_dict_ = X_last.groupby(["item_id"])["monthly_sale"].agg(['sum','mean']).to_dict(orient="index")
        return self
    
    def transform(self, X, y=None):
        last_month_item_mean = np.zeros(X.shape[0])
        last_month_item_total = np.zeros(X.shape[0])
        
        i = 0 
        for idx, row in X.iterrows():
            item_id = row["item_id"]
            if item_id in self.last_month_sales_dict_:
                last_month_item_mean[i] = self.last_month_sales_dict_[item_id]['mean']
                last_month_item_total[i] = self.last_month_sales_dict_[item_id]['sum']
            i+=1
            
        X["last_month_item_mean"] = last_month_item_mean
        X["last_month_item_total"] = last_month_item_total
    
        return X

In [ ]:
last_month_item_sale_transformer = LastMonthItemSaleTransformer()
last_month_item_sale_transformer.fit(all_data)

In [ ]:
all_data=last_month_item_sale_transformer.transform(all_data)

In [ ]:
all_data.head()

In [ ]:
test = last_month_item_sale_transformer.transform(test)

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
all_data.head()

### Train model

In [ ]:
all_data.shape

In [ ]:
train = all_data.loc[all_data["date_block_num"]==33]
train = train.drop(["date_block_num"],axis=1)

In [ ]:
train.shape

In [ ]:
train.sample()

In [ ]:
test.head()

In [ ]:
train.columns

In [ ]:
X_train = train.drop(["monthly_sale"],axis=1)
y_train = train["monthly_sale"].fillna(0).clip(0,20)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
y